## Project #4:

* Student name: Milena Afeworki
* Student pace: full time
* Scheduled project review date/time: 08/05/2021 @ 10:15 PT
* Instructor name: Abhineet Kulkarni 
* Blog post URL:

# Time series Prediction on California Home sales

With 54 of the Fortune 500 companies headquartered in California, like Google, Apple, Disney, Oracle and Intel among others, California is positioned for continued job growth. High employment rates draw renters and buyers and, for investors, enhance the likelihood of consistent cash flow. Though not the lowest in the country, California has favorable property tax rates, which will help control investors’ expenses and improve cash flow. The combination of job growth and a world-renowned lifestyle and culture supports home values. People buy where they want to live, and millions of people want to live in California. All of the demand mentioned above also leads to increasing home values.
 

## Business Problem

If a real estate company is looking to flip homes, what are the top 5 zip codes to invest in? How will we scale our data? What defines best?

For this dataset, we will only include data from CA. Because of the housing market crash, any modelling that uses only recent years may be misleading. We will use every value from 1996 to 2018 so we can have the most accurate picture of home values in CA through the years. 

## Data understanding

Zillow provides their users the opportunity to use their platform to access specific datasets for research purposes. The dataset that we will be using contains the median home sales prices throughout all states sorted by their zip codes. With this dataset we can extract a lot of insight through out all states with the potential to understand markets and develop investment strategies. This platform allows the public to do independent research in any market in the US.

This dataset contains 14723 rows and 272 columns.

# Load the Data/Filtering for Chosen Zipcodes

In [3]:
#Import libraries
import numpy as np
import folium
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from scipy import stats
from random import gauss as gs
import math
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pandas.plotting import autocorrelation_plot, lag_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')
import pmdarima as pm
from pmdarima import auto_arima
from matplotlib.pylab import rcParams
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
df = pd.read_csv('zillow_data.csv')
df.head()

In [ ]:
df.shape

# Data Preprocessing

Let's replace the column 'RegionName' by 'ZipCode' and then select only the data for California and drop the rest.

In [ ]:
#Rename RegionName
df.rename({'RegionName': 'ZipCode'}, axis='columns', inplace=True)

In [ ]:
#Delete all but CA zipcodes
df_ca = df.loc[df['State']== 'CA'].reset_index()
df_ca.drop(['index', 'RegionID', 'SizeRank'], axis=1, inplace=True)
print('Total Zipcodes in DataFrame:', len(df_ca))

In [ ]:
#Check for zeros
df_ca.describe()

# Reshape from Wide to Long Format

Our next step would be to change the format of the data frame from wide format to long format and index by the'Date' column 

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['ZipCode', 'City', 'State', 'Metro', 'CountyName'], 
                     var_name='Date')
    melted['Date'] = pd.to_datetime(melted['Date'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

In [ ]:
melted_df = melt_data(df_ca)

We will also want to make sure we change zip code into a string so it is not confused for an integer.

In [ ]:
#Change Zipcode dtype to 'str'
melted_df['ZipCode'] = melted_df['ZipCode'].astype(str)

# Make sure the data type of the 'Date' column is datetime
melted_df['Date'] = pd.to_datetime(melted_df['Date'], format='%m/%y')

# Set the 'Date' column as index
melted_df.set_index('Date', inplace=True)

In [ ]:
melted_df.index

In [ ]:
melted_df.head()

In [ ]:
melted_df.tail()

In [ ]:
melted_df.shape

# EDA and Visualization


On this step we will be creating visualizations to get a better idea of what we are working with and also to understand the trends of the values in our data.

In [ ]:
#check for nulls 
melted_df.isna().sum()

In [ ]:
metro = melted_df.groupby('Metro')
metro = metro.value.mean()
metro = metro.sort_values(ascending=False).head(10)

In [ ]:
county = melted_df.groupby('CountyName')
county = county.value.mean()
county = county.sort_values(ascending=False).head(10)

In [ ]:
fig =plt.figure(figsize=(8,5))
county.plot.barh(color='green')
plt.title('Average value of Home per county')
plt.xlabel('Home value')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,5))
metro.plot.barh(color='green')
plt.title('Average Value of Home by City')
plt.xlabel('Value ($)')
plt.show()


Here we get a good idea of the average home value per County name.

In [ ]:
print('Average CA home value' ,round(melted_df['value'].mean()))

In [ ]:
#data resampled by month
monthly_data = melted_df['value'].resample('MS').mean()
monthly_data = monthly_data.fillna(monthly_data.bfill())
monthly_data.plot(figsize=(12,5))
plt.title('Average CA Home Value By Month')
plt.ylabel('Value')
plt.show()
print(monthly_data.head())

In [ ]:
monthly_data['1996':'1999'].plot(figsize=(12,5))

In [ ]:
monthly_data['2006':].plot(figsize=(12,5))

In [ ]:
yearly_data = melted_df['value'].resample('A').mean()
yearly_data.plot.line(figsize=(12,5))
plt.title('Average CA Home Value By Year')
plt.ylabel('Value in US Dollars ($)')
plt.show()
print(yearly_data.head())

In [ ]:
fig = plt.figure( figsize=(8,5))
monthly_data.hist(color='green')

In [ ]:
fig = plt.figure( figsize=(8,5))
monthly_data.plot(kind='kde')
plt.legend()

The monthly and yearly home values seem ot be similar. There doesn't seem to be any seasonality but we do see a generaly upward trend with a dip downward between the years 2007-2012.
Next, we will cut down on variation to ensure we get the true most valuable zip codes.

In [ ]:
#taking into account the last 5years
df_ca.iloc[:,-60:].head()

In [ ]:
df_ca['yr_avg']=df_ca.iloc[:,-60:].mean(skipna=True, axis=1)

#Get zipcodes with an average value 2 decile above the median and 2 deciles below.
print(df_ca['yr_avg'].describe(),'\n')

#Calculate the 70% cutoff value (2 decile above).
q_70 = df_ca['yr_avg'].quantile(q=0.70)
print(f'Average Value 70% cutoff value: {round(q_70,2)}')

#Calculate the 30% cutoff value (2 deciles below).
q_30 = df_ca['yr_avg'].quantile(q=0.30)
print(f'Average Value 30% cutoff value: {round(q_30,2)}')

#Get data frame with selected zipcodes.
df_avg = df_ca[(df_ca['yr_avg']<q_70) & (df_ca['yr_avg']>q_30)]
print(f'Amount of zipcodes: {len(df_avg)}')

In [ ]:
df_avg.head()

In finance, the coefficient of variation allows investors to determine how much volatility, or risk, is assumed in comparison to the amount of return expected from investments. Ideally, if the coefficient of variation formula should result in a lower ratio of the standard deviation to mean return, then the better the risk-return trade-off. Therefore, in these next steps we are going to filter the data some more by calculating the CV value and only selecting values with in the company's risk factor (assume 60 percentile).

In [ ]:
#Calculate historical return on investment
df_avg['ROI'] = (df_avg['yr_avg']/df_avg['1996-04'])-1

#Calculate standard deviation of monthly values
df_avg['std'] = df_avg.loc[:,'1996-04':'2018-04'].std(skipna=True, axis=1)

#Calculate historical mean value
df_avg['mean'] = df_avg.loc[:,'1996-04':'2018-04'].mean(skipna=True, axis=1)

#Calculate coefficient of variation
df_avg['CV'] = df_avg['std']/df_avg['mean']

#Show calculated values
df_avg[['ZipCode','std','mean','ROI','CV','CountyName']].head()

In [ ]:
#find out the top 10 couties with highest ROI
grp_county = df_avg.groupby('CountyName', group_keys=False).sum()['ROI']
grp_county.sort_values(ascending=False)[:10]

# sorted(round(grouped_county,2), reverse=True)[:10]

In [ ]:
#top 10 counties with highest ROI before considering risk factor CV
grp_county.sort_values(ascending=False)[:10].keys() 

In [ ]:
#Descriptive statistics of coefficients of variance.
print(df_avg.CV.describe())

#Define upper limit of CV according to risk profile.
upper_cv = df_avg.CV.quantile(.6)
print(f'\nCV upper limit: {upper_cv}')

#Get the 10 counties with highest ROIs within the firms risk profile.
df_top10 = df_avg[df_avg['CV']<upper_cv].sort_values('ROI', axis=0, ascending=False)

#find out the top 10 couties with highest ROI
grp_county = df_top10.groupby('CountyName').sum()['ROI']
grp_county.sort_values(ascending=False)[:10]

Now for each county lets look into the zipcode with the highest ROI value and move onto the time series analysis


In [ ]:
top10_county = list(grp_county.sort_values(ascending=False)[:10].index)
top10_county

In [ ]:
df_top10.shape

In [ ]:
df_top10 = df_top10.loc[df_top10['CountyName'].isin(top10_county)]
df_top10.shape

In [ ]:
df_top10['CountyName'].value_counts()

In [ ]:
df_top10.groupby('CountyName').max()['ROI']

In [ ]:
df_top10.isna().sum()

In [ ]:
#Get city and state names for each zip code
ziplist = []
top_ROI = {}

for i in top10_county:
    City = df_top10[df_top10['CountyName']==i].City.values[0]
    Metro = df_top10[df_top10['CountyName']==i].Metro.values[0]
    Zipcode = df_top10[df_top10['CountyName']==i].ZipCode.values[0]
    roi = (df_top10[df_top10['CountyName']==i].max()['ROI'])*100
    
    ziplist.append(Zipcode)
    top_ROI[i] = roi
    print(f'County: {i} \nCity: {City}, Zipcode: {Zipcode}, Metro: {Metro}\n')


# Time Series Analysis

In [ ]:
ziplist

In [ ]:
ziplist = ['92101', '91754', '92866', '92860', '96141', '95441', '93405',
           '95818', '93003', '94546']

In [ ]:
x = dict(sorted(top_ROI.items(), key=lambda item: item[1])).keys()

In [ ]:
y = dict(sorted(top_ROI.items(), key=lambda item: item[1])).values()

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(x, y, color='green', )
plt.title('ROI of Homes in Top 10 CA Counties')
plt.xlabel('County Name')
plt.ylabel('% Gained on Original Investment')
plt.show()

In [ ]:
#create a dictionary for each zipcode
ts = {}
for zc in ziplist:
    temp_df = melted_df.groupby('ZipCode').get_group(zc).sort_index()['value']
    ts[zc] = temp_df

In [ ]:
ts

In [ ]:
ts_df = pd.DataFrame(ts)
ts_df.head()

In [ ]:
zip_1 = ziplist[0]

In [ ]:
ts_zip1 = ts_df[zip_1].copy()
ax = ts_zip1.plot()
ax.legend()
plt.show()

## Model 1

### Baseline Model

In [ ]:
# selected params
d = 1
p = 1
q = 1

In [ ]:
train_size = 0.85 #leaving approximately 3year for test size.
split_idx = round(len(ts_zip1)* train_size)
split_idx

## Split
train = ts_zip1.iloc[:split_idx]
test = ts_zip1.iloc[split_idx:]

## Visualize split
fig,ax= plt.subplots()
kws = dict(ax=ax,marker='o')
train.plot(**kws)
test.plot(**kws)
ax.legend(bbox_to_anchor=[1,1])
plt.show()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

## Baseline model from eye-balled params
model = SARIMAX(train,order=(p,d,q),).fit()
display(model.summary())
model.plot_diagnostics(figsize=(10,8));
plt.show()

In [ ]:
## obtaining forecast
from sklearn import metrics
forecast = model.get_forecast(steps=len(test))

In [ ]:
def forecast_to_df(forecast,zipcode):
    test_pred = forecast.conf_int()
    test_pred[zipcode] = forecast.predicted_mean
    test_pred.columns = ['lower','upper','prediction']
    return test_pred

pred_df = forecast_to_df(forecast,zip_1)
pred_df.head()

In [ ]:
def plot_train_test_pred(train,test,pred_df):
    fig,ax = plt.subplots()
    kws = dict(marker='o')
    
    ax.plot(train,label='Train',**kws)
    ax.plot(test,label='Test',**kws)
    ax.plot(pred_df['prediction'],label='prediction',ls='--',**kws)

    ax.fill_between(x=pred_df.index,y1=pred_df['lower'],y2=pred_df['upper'])
    ax.legend(bbox_to_anchor=[1,1])
    fig.tight_layout()
    return fig,ax

In [ ]:
plot_train_test_pred(train,test,pred_df)

plt.show()


## Model 2 

In [ ]:
auto_model = auto_arima(train,start_p=0,start_q=0)
display(auto_model.summary())
auto_model.plot_diagnostics(figsize=(10,8));

In [ ]:
model3 = SARIMAX(ts_zip1,order=auto_model.order,
                     seasonal_order=auto_model.seasonal_order).fit()
display(model3.summary())
model3.plot_diagnostics(figsize=(10,8));

In [ ]:
pred = model3.get_forecast(steps=12)#start=test.index[0],end=test.index[-1])
pred_df = forecast_to_df(pred,zip_1)
display(plot_train_test_pred(train,test,pred_df));
plt.show()

In [ ]:
  ## Get predictions
pred  = model3.get_forecast(steps=36)#start=test.index[0],end=test.index[-36])
pred_df = forecast_to_df(pred,zc)

# Get the real and predicted values
output = model3.get_prediction(start='2015-01',end='2018-04', dynamic=True)
value_forcasted = output.predicted_mean
print('Predicted mean budget: ', round(value_forcasted.max(), 1))
value_truth = test[:]

train_pred = model3.get_prediction(start='1996-04',end='2014-12')
train_forcast = train_pred.predicted_mean
train_true = train[:]
print('predicted', train_pred)
print('forcasted', train_forcast)
# print('predicted', train_pred)
print('forcasted', value_forcasted)

 # Compute the root mean square error for train set
rmse_train = np.sqrt(metrics.mean_squared_error(train_true, train_forcast))

print('SARIMA model RMSE on train data: {}'.format(round(rmse_train, 1)))

# Compute the root mean square error for test set
mse = ((value_forcasted - value_truth) ** 2).mean()
rmse = sqrt(mse)
print('SARIMA model RMSE on test data: {}'.format(round(rmse, 1)))


In [ ]:
RESULTS = {}

for zc in ziplist:
    print(zc)
    
    ## Make empty dict for district data
    zipcode_d = {}
    
    ## Copy Time Series
    ts_final = ts_df[zc].copy()
    
    ## Train Test Split Index
    train_size = 0.85
    split_idx = round(len(ts_df)* train_size)

    ## Split
    train = ts_final.iloc[:split_idx]
    test = ts_final.iloc[split_idx:]
    
    
    ## Get best params using auto_arima
    gridsearch_model = auto_arima(ts_final,start_p=0,start_q=0)
    model3 = SARIMAX(ts_final,order=gridsearch_model.order,
                     seasonal_order=gridsearch_model.seasonal_order).fit()
    
    ## Get predictions
    pred  = model3.get_forecast(steps=36)#start=test.index[0],end=test.index[-36])
    pred_df = forecast_to_df(pred,zc)

    # Get the real and predicted values
    output = model3.get_prediction(start='2015-01',end='2018-04', dynamic=True)
    value_forcasted = output.predicted_mean
    print('Predicted mean budget: ', round(value_forcasted.max(), 1))
    value_truth = test[:]
    
    train_pred = model3.get_prediction(start='1996-04',end='2014-12')
    train_forcast = train_pred.predicted_mean
    train_true = train[:]
    
     # Compute the root mean square error for train set
#     mse_train = ((train_forcast - train_true) ** 2).mean()
#     rmse_train = math.sqrt(mse_train)
    rmse_train = np.sqrt(metrics.mean_squared_error(train_true, train_forcast))
    
    print('SARIMA model RMSE on train data: {}'.format(round(rmse_train, 1)))
    
    # Compute the root mean square error for test set
    mse = ((value_forcasted - value_truth) ** 2).mean()
    rmse = sqrt(mse)
    print('SARIMA model RMSE on test data: {}'.format(round(rmse, 1)))

    ## Save info to dict
    zipcode_d['pred_df'] = pred_df
    zipcode_d['model'] = model3
    zipcode_d['train'] = train
    zipcode_d['test'] = test
    
    ## Display Results
    display(model3.summary())
    plot_train_test_pred(train,test,pred_df)
    plt.xlabel('Year')
    plt.ylabel('Value in US Dollars ($)')
    plt.show()
    
    
    ## Save district dict in RESULTS
    RESULTS[zc] = zipcode_d
    print('---'*20,end='\n\n')

From the predictions, we can see that the best zip codes are 96141, 95441, 92101, 93405, and 91754 since those are with the lowest number of parameters from the summary. Let's check for stationarity and measure their future returns.

In [ ]:

zip_96141 = melted_df[melted_df.ZipCode == '96141']
zip_93405 = melted_df[melted_df.ZipCode == '93405']
zip_92866 = melted_df[melted_df.ZipCode == '92866']
zip_92101 = melted_df[melted_df.ZipCode == '92101']
zip_95441 = melted_df[melted_df.ZipCode == '95441']
zip_94546 = melted_df[melted_df.ZipCode == '94546']
zip_91754 = melted_df[melted_df.ZipCode == '91754']
zip_92860 = melted_df[melted_df.ZipCode == '92860']
zip_95818 = melted_df[melted_df.ZipCode == '95818']
zip_93003 = melted_df[melted_df.ZipCode == '93003']

In [ ]:
zip_df = pd.DataFrame()
zip_df = zip_df.append(zip_96141)
zip_df = zip_df.append(zip_93405)
zip_df = zip_df.append(zip_92866)
zip_df = zip_df.append(zip_92101)
zip_df = zip_df.append(zip_95441)
zip_df = zip_df.append(zip_94546)
zip_df = zip_df.append(zip_91754)
zip_df = zip_df.append(zip_92860)
zip_df = zip_df.append(zip_95818)
zip_df = zip_df.append(zip_93003)
zip_df.head()

In [ ]:
zip_ts = []
for zc in zip_df.ZipCode.unique():
    #Create separate dataframes for each zipcode with a monthly frequency.
    top5_df = zip_df[zip_df['ZipCode']==zc].asfreq('MS')
    zip_ts.append(top5_df)
zip_ts[0].head()

In [ ]:
#checking how much each zipcode was impacted during the recession
for i in range(len(zip_ts)):
    print(zip_ts[i].ZipCode[0])
    print(zip_ts[i]['2006-01':'2011-01'].value) 
    print('--------------------------')

In [ ]:
for i in range(len(zip_ts)):
    print(f'Value descriptive statistics for zipcode {zip_ts[i].ZipCode[0]}:')
    print(f'{zip_ts[i].value.describe()}\n')

In [ ]:
for i in range(10):
    zip_ts[i].value.plot(label=zip_ts[i].ZipCode[0],figsize=(15,6))
    plt.title('Average Home values of each zipcode')
    plt.legend()

## Decomposition

In [ ]:
# Import and apply seasonal_decompose()
def decompose(i):
    from statsmodels.tsa.seasonal import seasonal_decompose
    print('Zip code:', zip_ts[i]['ZipCode'][1])
    decomposition = seasonal_decompose(zip_ts[i]['value'])

    # Gather the trend, seasonality, and residuals 
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    # Plot gathered statistics
    plt.figure(figsize=(12,8))
    plt.subplot(411)
    plt.plot(zip_ts[i]['value'], label='Original', color='blue')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(trend, label='Trend', color='blue')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(seasonal,label='Seasonality', color='blue')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(residual, label='Residuals', color='blue')
    plt.legend(loc='best')
    plt.tight_layout()

In [ ]:
decompose(0)

## Checking Stationarity

In [ ]:
#Calculate monthly returns in new column 'ret' for each zipcode.
for zc in range(len(zip_ts)):
    zip_ts[zc]['ret']=np.nan*len(zip_ts[zc])
    for i in range(len(zip_ts[zc])-1):
        zip_ts[zc]['ret'][i+1]= (zip_ts[zc].value.iloc[i+1] / zip_ts[zc].value.iloc[i]) - 1

In [ ]:
for i in range(10):
    results = adfuller(zip_ts[i].ret.dropna())
    print(f'ADFuller test p-value for zipcode: {zip_ts[i].ZipCode[0]}')
    print('p-value:',results[1])
    if results[1]>0.05:
        print('Fail to reject the null hypothesis. Data is not stationary.\n')
    else:
        print('Reject the null hypothesis. Data is stationary.\n')

In [ ]:
results

Except for the last zipcode we fail to reject the null hypothesis that the data is not stationary.

In [ ]:
roll_mean = top5_df['value'].rolling(window=8, center=False).mean()
roll_std = top5_df['value'].rolling(window=8, center=False).std()
# series_ts = pd.Series()

In [ ]:
fig = plt.figure(figsize=(12,7))
plt.plot(top5_df['value'], color='blue', label='Original')
plt.plot(roll_mean, color='red', label='Rolling Mean')
plt.plot(roll_std, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

In [ ]:
for i in range(10):
    #Perform adfuller test and drop NaN values created when calculating monthly returns.
    results = adfuller(zip_ts[i].ret.diff().dropna()) #differencing by 12 month for stationarity
    print(f'ADFuller test p-value for zipcode: {zip_ts[i].ZipCode[0]}')
    print('p-value:',results[1])
    if results[1]>0.05:
        print('Fail to reject the null hypothesis. Data is not stationary.\n')
    else:
        print('Reject the null hypothesis. Data is stationary.\n')

In [ ]:
results

In [ ]:
zip_ts

In [ ]:
#Create individual time series for each of the positive zipcodes
TS_96141 = zip_ts[0].ret.dropna()
TS_96141d = zip_ts[0].ret.diff().dropna()

TS_93405 = zip_ts[1].ret.dropna()
TS_93405d = zip_ts[1].ret.diff().dropna()

TS_92866 = zip_ts[2].ret.dropna()
TS_92866d = zip_ts[2].ret.diff().dropna()

TS_92101 = zip_ts[3].ret.dropna()
TS_92101d = zip_ts[3].ret.diff().dropna()

TS_95441 = zip_ts[4].ret.dropna()
TS_95441d = zip_ts[4].ret.diff().dropna()

TS_94546 = zip_ts[5].ret.dropna()
TS_94546d = zip_ts[5].ret.diff().dropna()

TS_91754 = zip_ts[6].ret.dropna()
TS_91754d = zip_ts[6].ret.diff().dropna()

TS_92860 = zip_ts[7].ret.dropna()
TS_92860d = zip_ts[7].ret.diff().dropna()

TS_95818 = zip_ts[8].ret.dropna()
TS_95818d = zip_ts[8].ret.diff().dropna()

TS_93003 = zip_ts[9].ret.dropna()
TS_93003d = zip_ts[9].ret.diff().dropna()


In [ ]:
def plot_acf_pacf(ts, figsize=(10,8),lags=24):
    
    fig,ax = plt.subplots(nrows=3, figsize=figsize)
    
    ts.plot(ax=ax[0])
    
    plot_acf(ts,ax=ax[1],lags=lags)
    plot_pacf(ts, ax=ax[2],lags=lags) 
    fig.tight_layout()
    
    
    for a in ax[1:]:
        a.xaxis.set_major_locator(mpl.ticker.MaxNLocator(min_n_ticks=lags, integer=True))
        a.xaxis.grid()
    return fig,ax

def seasonal_plots(df,N=4,lags=[12,24,36,48,60,72]):
    #Differencing the rolling mean to find seasonality in the resulting acf plot.
    fig,(ax1,ax2) = plt.subplots(2,1,figsize=(13,8))
    rolling = df - df.rolling(N).mean()
    plot_acf(rolling.dropna(),lags=lags,ax=ax1)
    plot_pacf(rolling.dropna(),lags=lags,ax=ax2)
    plt.show();

def model_fit(df,pdq=(1,0,1),pdqs=(0,0,0,1)):
    train, test = train_test(df)
    model = SARIMAX(train,order=pdq,seasonal_order=pdqs)
    results = model.fit()
    results.summary
    residuals = results.resid
    print(results.summary())
    results.plot_diagnostics(figsize=(11,8))
    plt.show();
    return train, test, results

def forecast_model(df,pdq=(1,0,1),pdqs=(0,0,0,12), display=True,zc='input zipcode'):
    model = SARIMAX(df, order=pdq,seasonal_order=pdqs)
    model_fit = model.fit()
    output = model_fit.get_prediction(start='2018-04',end='2028-04', dynamic=True)

    forecast_ci = output.conf_int()
    if display:
        fig, ax = plt.subplots(figsize=(13,6))
        output.predicted_mean.plot(label='Forecast')
        ax.fill_between(forecast_ci.index,forecast_ci.iloc[:, 0],forecast_ci.iloc[:, 1],
                        color='k', alpha=.25,label='Conf Interval')
        plt.title('Forecast of Monthly Returns')
        plt.xlabel('Time')
        plt.legend(loc='best')
        plt.show()
#     year_1= (1+output.predicted_mean[:12]).prod()-1
    year_1= (1+output.predicted_mean[:12]).prod()-1
    year_3=(1+output.predicted_mean[:36]).prod()-1
    year_5= (1+output.predicted_mean[:60]).prod()-1
    year_10=(1+output.predicted_mean).prod()-1
    print(f'Total expected return in 1 year: {round(year_1*100,2)}%')
    print(f'Total expected return in 3 years: {round(year_3*100,2)}%')
    print(f'Total expected return in 5 year: {round(year_5*100,2)}%')
    print(f'Total expected return in 10 years: {round(year_10*100,2)}%')
    tot_ret = [zc,year_1,year_3,year_5,year_10]
    return tot_ret

# X Zipcode 96141: Placer county

In [ ]:
plot_acf_pacf(TS_96141d,lags=20);

Even though the data lines after differencing do seem to be fluctuating, the movements seem to be completely random, 
and the same conclusion holds for the original time series.

In [ ]:
results = pm.auto_arima(TS_96141d,information_criterion='aic',m=12,
                        start_p=0,start_q=0, max_p=3, max_q=3,
                        stepwise=True,trace=True,error_action='ignore',suppress_warnings=True)
results

In [ ]:
pdq = (2, 0, 3)
pdqs = (0, 0, 2, 12)
ret_96141 = forecast_model(TS_96141,pdq=pdq,pdqs=pdqs,zc=96141)

# Zipcode 93405: San Luis Obispo

In [ ]:
plot_acf_pacf(TS_93405d,lags=20)

The ACF and PACF have just one very strong correlation, right at 1 month.

In [ ]:
results = pm.auto_arima(TS_93405d,information_criterion='aic',m=12,d=0,
                        start_p=1,start_q=1, max_p=3, max_q=3,
                        stepwise=True,trace=True,error_action='ignore',suppress_warnings=True)
results

In [ ]:
pdq = (2, 0, 3)
pdqs = (1, 0, 1, 12)
ret_93405 = forecast_model(TS_93405,pdq=pdq,pdqs=pdqs,zc=90504)

# Zipcode 92866: Los Angeles-Long Beach-Anaheim

In [ ]:
plot_acf_pacf(TS_92866d,lags=20)

In [ ]:
results = pm.auto_arima(TS_92866d,information_criterion='aic',m=12,d=0,
                        start_p=1,start_q=1, max_p=3, max_q=3,
                        stepwise=True,trace=True,error_action='ignore',suppress_warnings=True)
results

In [ ]:
pdq = (0, 0, 1)
pdqs = (0, 0, 1, 12)
ret_92866 = forecast_model(TS_92866,pdq=pdq,pdqs=pdqs,zc=92866)

# Zipcode 92101: San Diego

In [ ]:
plot_acf_pacf(TS_92101d,lags=20)

The ACF and PACF have just one very strong correlation, right at 2 month.

In [ ]:
results = pm.auto_arima(TS_92101d,information_criterion='aic',m=12,d=0,
                        start_p=1,start_q=1, max_p=3, max_q=3,
                        stepwise=True,trace=True,error_action='ignore',suppress_warnings=True)
results

In [ ]:
pdq = (2, 0, 1)
pdqs = (1, 0, 2, 12)
ret_92101 = forecast_model(TS_92101,pdq=pdq,pdqs=pdqs,zc=92101)

# X Zipcode 95441: Sonoma county

In [ ]:
plot_acf_pacf(TS_95441d, lags=20)

The ACF and PACF have just one very strong correlation, right at 2 month.

In [ ]:
results = pm.auto_arima(TS_95441d,information_criterion='aic',m=12,d=0,
                        start_p=1,start_q=1, max_p=3, max_q=3,
                        stepwise=True,trace=True,error_action='ignore',suppress_warnings=True)
results

In [ ]:
pdq = (2, 0, 2)
pdqs = (2, 0, 2, 12)
ret_93405 = forecast_model(TS_95441,pdq=pdq,pdqs=pdqs,zc=93405)

# X Zipcode 94546: San Francisco

In [ ]:
plot_acf_pacf(TS_94546d,lags=20)

In [ ]:
results = pm.auto_arima(TS_94546d,information_criterion='aic',m=12,d=0,
                        start_p=1,start_q=1, max_p=3, max_q=3,
                        stepwise=True,trace=True,error_action='ignore',suppress_warnings=True)
results

In [ ]:
pdq = (3, 0, 0)
pdqs = (0, 0, 2, 12)
ret_94546 = forecast_model(TS_94546,pdq=pdq,pdqs=pdqs,zc=94546)

# Zipcode 91754: Los Angeles

In [ ]:
plot_acf_pacf(TS_91754d,lags=20)

The ACF and PACF have just one very strong correlation, right at 1 month.

In [ ]:
results = pm.auto_arima(TS_91754d,information_criterion='aic',m=12,d=0,
                        start_p=1,start_q=1, max_p=3, max_q=3,
                        stepwise=True,trace=True,error_action='ignore',suppress_warnings=True)
results

In [ ]:
pdq = (2, 0, 1)
pdqs = (0, 0, 0, 12)
ret_91754 = forecast_model(TS_91754,pdq=pdq,pdqs=pdqs,zc=91754)

# Zipcode 92860: Riverside

In [ ]:
plot_acf_pacf(TS_92860d,lags=20)

In [ ]:
results = pm.auto_arima(TS_92860d,information_criterion='aic',m=12,d=0,
                        start_p=1,start_q=1, max_p=3, max_q=3,
                        stepwise=True,trace=True,error_action='ignore',suppress_warnings=True)
results

In [ ]:
pdq = (2, 0, 2)
pdqs = (0, 0, 1, 12)
ret_92860 = forecast_model(TS_92860,pdq=pdq,pdqs=pdqs,zc=92860)

# X Zipcode 95818: Sacramento

In [ ]:
plot_acf_pacf(TS_95818d,lags=20)

In [ ]:
results = pm.auto_arima(TS_95818d,information_criterion='aic',m=12,d=0,
                        start_p=1,start_q=1, max_p=3, max_q=3,
                        stepwise=True,trace=True,error_action='ignore',suppress_warnings=True)
results

In [ ]:
pdq = (3, 0, 2)
pdqs = (2, 0, 0, 12)
ret_95818 = forecast_model(TS_95818,pdq=pdq,pdqs=pdqs,zc=95818)

# X Zipcode 93003: Ventura

In [ ]:
plot_acf_pacf(TS_93003d,lags=20)

In [ ]:
results = pm.auto_arima(TS_93003d,information_criterion='aic',m=12,d=0,
                        start_p=1,start_q=1, max_p=3, max_q=3,
                        stepwise=True,trace=True,error_action='ignore',suppress_warnings=True)
results

In [ ]:
pdq = (0, 0, 1)
pdqs = (0, 0, 0, 12)
ret_93003 = forecast_model(TS_93003,pdq=pdq,pdqs=pdqs,zc=93003)

## Conclusion and Recommendation


For the Real estate looking to immediately invest in the following zipcodes, here are the recommendations on the budget worth of a home and whether it is advisable to buy ,flip and sell the house, or buy and hold.


**Zip code 92866 (LA- Long Beach county):** Buy, Flip and sell homes within a year. (Budget of $584,000)

                                
                            Total expected return in 1 year: 2.5%
                            Total expected return in 3 years: 2.67%
                            Total expected return in 5 year: 2.67%
                            Total expected return in 10 years: 2.67%
                                
                                
**Zip code 93405 (San Luis Obispo):** Buy and hold for the next 5-10 years. (Budget of $642,000)

                                  Total expected return in 1 year: 8.1%
                                  Total expected return in 3 years: 12.39%          
                                  Total expected return in 5 year: 13.0%
                                  Total expected return in 10 years: 13.13% 
                                  
                                  
**Zip code 92101 (San Diego county):** Buy and hold for the next 3-5 years. (Budget of $552,000)

                                   Total expected return in 1 year: 10.47%
                                   Total expected return in 3 years: 14.06%
                                   Total expected return in 5 year: 14.27%
                                   Total expected return in 10 years: 14.27%

                            
**Zip code 92860 (Riverside County):** Buy, flip and sell within a year. (Budget of $439,000)

                            Total expected return in 1 year: 7.43%
                            Total expected return in 3 years: 11.23%
                            Total expected return in 5 year: 11.82%
                            Total expected return in 10 years: 11.93%

**Zip code 91754 (Los Angeles):** Buy and hold for atleast 10years. (Budget of $587,000)

                            Total expected return in 1 year: 2.6%
                            Total expected return in 3 years: 4.72%
                            Total expected return in 5 year: 5.32%
                            Total expected return in 10 years: 5.54%

       



## Further study

- The model is unable to correctly adjust to unique events such as exogenous data. Interest rates, rent values and GDP would be important factors to explore the relationship they would have with the home values. Rent income should exceed the costs of maintenance, mortgage, insurance, taxes and other expenses. Any gains that may be realized from selling the property later should also be factored into the calculation.


- Model would be more effective with more recent years data and considering the impact of recent events on Real Estate investment.

In [4]:
# Import the pandas library
import pandas as pd

# Make an empty map
m = folium.Map(location=[36,-120], tiles="OpenStreetMap", zoom_start=6)

# Show the map
m

# Make a data frame with dots to show on the map
data = pd.DataFrame({
   'lon':[-120.6423, -120.6636, -117.0262, -116.0358, -118.2371],
   'lat':[39.1308, 35.2715, 32.5655, 33.7326, 34.1381],
   'name':['Placer county', 'San Luis Obispo', 'San Diego', ' Riverside', 'Los Angeles'],
   'value':[583638, 641957, 551370, 439238, 587200]
}, dtype=str)

data

,lon,lat,name,value
0,-120.6423,39.1308,Placer county,583638
1,-120.6636,35.2715,San Luis Obispo,641957
2,-117.0262,32.5655,San Diego,551370
3,-116.0358,33.7326,Riverside,439238
4,-118.2371,34.1381,Los Angeles,587200


In [5]:
for i in range(0,len(data)):
    folium.Marker(location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
     popup=folium.Popup(data.iloc[i]['name'], show=True),
   ).add_to(m)
    
m
